In [1]:
from tkinter import *
from tkinter import filedialog
import pandas as pd
import pyodbc
import linecache
import snowflake.connector
import requests
import os
import copy
import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=DeprecationWarning)

C:\Users\sxm0326\AppData\Roaming\Python\Python310\site-packages\snowflake\connector\options.py:96: UserWarning: You have an incompatible version of 'pyarrow' installed (12.0.1), please install a version that adheres to: 'pyarrow<6.1.0,>=6.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [2]:
df_bom = pd.DataFrame()
df_exploded_bom = pd.DataFrame()
orig_df_exploded_bom = pd.DataFrame()
df_bom_cost = pd.DataFrame()
df_exploded_bom_cost = pd.DataFrame()
df_prod_dim = pd.DataFrame()

In [3]:
def save_data(df, file_name):
    """
    This function saves the content of the input dataframe to a file, specified by the parameter file_name.
    """
    df.to_csv(file_name, index = False)

In [4]:
def choose_file():
    """
    This function allows a dialog box to select the file. 
    """
    global df_bom
    filepath = filedialog.askopenfilename(initialdir = "C:\\Users\\sxm0326\\Download",
                                          title = "Select a text file...",
                                          filetypes = (("text files", "*.txt"),
                                          ("all files", "*.*")))
    bom_file = open(filepath,'r')
    df_bom = pd.read_csv(bom_file)
    bom_file.close()

In [5]:
def close_window():
    """
    This function closes the dialog box.
    """
    window.destroy()

In [6]:
def upload_file():
    """
    This function uploads a text file. 
    Once the file upload is done, using the Exit button, the dialog box can be closed.
    """
    global window
    window = Tk()
    #Set the geometry of tkinter frame
    window.geometry("750x250")
    button = Button(text = "Upload a text file", command = choose_file)
    button.pack()
    
    # Button for closing
    exit_button = Button(window, text = "Exit", command = close_window)
    exit_button.pack(pady = 20)

    window.mainloop()

In [7]:
upload_file()

In [8]:
def build_explode_bom_query():
    """
    This function builds the query to include parent and the component items.
    """
    global df_bom
    
    consolidated_query_string = ""
    for material in df_bom['bom_material'].values:
        sql = f"""
               SELECT zerolevelbom
                    , parentbom
                    , component
                    , bomitemnodenbr
                    , plant
                    , requiredqty
                    , qtyperbaseunit
                    , baseunitofmeasure
                    , priceunit
                    , standardprice 
               FROM   MFGRPT.dbo.tblBillOfMaterial
               WHERE   zerolevelbom IN (SELECT 
                                        DISTINCT parentbom
                                        FROM     MFGRPT.dbo.tblBillOfMaterial
                                        WHERE    zerolevelmaterial = {material} 
                                        AND      TRIM(baseunitofmeasure) <> ''
                                       ) 
               UNION ALL
               """.replace("\t"," ").replace("\n"," ")
        consolidated_query_string += sql
    consolidated_query_string = "SELECT DISTINCT x.* FROM (" + consolidated_query_string[:-34] + ") x ORDER BY plant, parentbom, component;"
    return consolidated_query_string

In [9]:
def get_exploded_bom():
    """
    This function generates the exploded BOM and results are saved in exploded_bom.csv.
    The BOM information is fetched from SQL Server database.
    """
    global df_exploded_bom
    global orig_df_exploded_bom
    
    conn_str = (
    'DRIVER={SQL Server};'
    'SERVER=SDN-P-SQLHA04,4204;'
    'DATABASE=MFGRPT;'
    'Trusted_Connection=yes;'
    )
    conn = pyodbc.connect(conn_str)
    cur = conn.cursor()
    df_exploded_bom = pd.read_sql(build_explode_bom_query(), conn)

    orig_df_exploded_bom = copy.deepcopy(df_exploded_bom)
    df_exploded_bom = df_exploded_bom[df_exploded_bom.baseunitofmeasure != '']
    save_data(df = df_exploded_bom, file_name = 'exploded_bom.csv')
    conn.close()

In [10]:
get_exploded_bom()

C:\Users\sxm0326\AppData\Local\Temp\ipykernel_31164\871759378.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_exploded_bom = pd.read_sql(build_explode_bom_query(), conn)


In [11]:
'''
Obtain credentials from credentials.txt where 1st line indicates user id, 
2nd line shows password and 3rd line is associated email address.

'''
userid, password, useremail = linecache.getline('credentials.txt', 1).strip(),  linecache.getline('credentials.txt', 2).strip(),  linecache.getline('credentials.txt', 3).strip()

In [12]:
def get_oauth_token(svc_username=None, svc_password=None):
    """
    Retrives the authentication token for SBD Snowflake.
    This function automatically requests for username and password from the user through interactive prompts.
    If using service account credentials, they can be passed as svc_username and svc_password.
    Keywords:
    svc_username -- Service account username for which the token should be provided.
    svc_password -- Password corresponding to the service account user.
    Example usage:
    # Authentication with your personal user id.
    from sbd_common_utils.snowflake_utils import get_oauth_token
    access_token = get_oauth_token()
    # Authentication with service account.
    from sbd_common_utils.snowflake_utils import get_oauth_token
    from sbd_common_utils.common_utils import get_service_account_creds
    username, password = get_service_account_creds("/datascience/sandbox/someapp/service-account")
    access_token = get_oauth_token(username, password)
    """

    if svc_username and svc_password:
        username = svc_username
        password = svc_password
    else:
        None


    r = requests.post(
        "https://ssoprod.sbdinc.com/as/token.oauth2",
        data={
            "client_id": "Snowflake",
            "grant_type": "password",
            "username": username,
            "password": password,
            "client_secret": 'f9sq630wmLP6UjpSsOk7kTuP6xccCrSOC4YhE1VdTq3GCupqR7gjYcpuhEGRJ9e0',
            "scope": "session:role-any",
        },
    )
    r.raise_for_status()
    access_token = r.json()["access_token"]
    return access_token

In [13]:
def build_bom_cost_query(df, component_column):
    """
    This function builds the query for the cost components associated with the items.
    """
    global df_exploded_bom
    consolidated_query_string = ""
    for material in df[component_column].values:
        sql = f"""
               SELECT prod.prod_key,
                      cst.prod_cost_key,
                      cst.src_sys_key,
                      cst.src_rcrd_create_dte,
                      cst.src_rcrd_create_userid,
                      cst.src_rcrd_upd_dte,
                      cst.src_rcrd_upd_userid,
                      cst.eff_dte,
                      cst.loc_key,
                      cst.prod_key,
                      cst.crncy_key,
                      cst.cost_eff_from_dte,
                      cst.cost_eff_to_dte,
                      cst.cost_ver,
                      cst.valtn_variant,
                      cst.cost_entr_flag,
                      cst.valtn_area_lkey,
                      cst.ctrl_area_lkey,
                      cst.prod_cost_lot_sz,
                      cst.valtn_uom_lkey,
                      cst.costed_by_user,
                      cst.addtv_cost_est_by_user,
                      cst.cost_stat_lkey,
                      cst.cost_variant_lkey,
                      cst.post_prd,
                      cst.post_yr,
                      cst.profit_cntr_lkey,
                      cst.exch_rate_typ,
                      cst.lcrncy_key,
                      cst.cost_est_ord_bom_flag,
                      cst.cost_cmpnt_strc,
                      cst.cost_run_name,
                      cst.cost_run_dte,
                      cst.crncy_type,
                      cst.prod_typ_lkey,
                      cst.fullcost_1   AS material,
                      cst.fullcost_2   AS interplant_component,
                      cst.fullcost_3   AS freight_absorption,
                      cst.fullcost_4   AS material_adjustment1,
                      cst.fullcost_5   AS material_adjustment2,
                      cst.fullcost_6   AS labor,
                      cst.fullcost_7   AS labor_adjustment,
                      cst.fullcost_8   AS overhead,
                      cst.fullcost_9   AS purchasing_overhead,
                      cst.fullcost_10  AS "bud credit tooling",
                      cst.fullcost_11  AS misc_supplies,
                      cst.fullcost_12  AS overhead_adjustment,
                      cst.fullcost_13  AS material_waste,
                      cst.fullcost_14  AS cash_discounts,
                      cst.fullcost_15  AS sub_contracting,
                      cst.fullcost_16  AS external_processing,
                      cst.fullcost_17  AS fixed_ph_support,
                      cst.fullcost_18  AS duty,
                      cst.fullcost_20,
                      cst.fullcost_30,
                      cst.fullcost_40,
                      cst.fullcost_50,
                      cst.fullcost_60,
                      cst.fullcost_65,
                      cst.fullcost_70,
                      cst.fullcost_80,
                      cst.fullcost_90,
                      cst.fullcost_95,
                      cst.fullcost_99,
                      cst.fixedcost_1,
                      cst.fixedcost_2,
                      cst.fixedcost_3,
                      cst.fixedcost_4,
                      cst.fixedcost_5,
                      cst.fixedcost_6,
                      cst.fixedcost_7,
                      cst.fixedcost_8,
                      cst.fixedcost_9,
                      cst.fixedcost_10,
                      cst.fixedcost_11,
                      cst.fixedcost_12,
                      cst.fixedcost_13,
                      cst.fixedcost_14,
                      cst.fixedcost_15,
                      cst.fixedcost_16,
                      cst.fixedcost_17,
                      cst.fixedcost_18,
                      cst.fixedcost_20,
                      cst.fixedcost_30,
                      cst.fixedcost_40,
                      cst.fixedcost_50,
                      cst.fixedcost_60,
                      cst.fixedcost_65,
                      cst.fixedcost_70,
                      cst.fixedcost_80,
                      cst.fixedcost_90,
                      cst.fixedcost_95,
                      cst.fixedcost_99,
                      cst.full_cost,
                      cst.fixed_cost

               FROM   (SELECT   *
                       FROM     "PROD_EDW"."CONSOLIDATED"."VW_EDW_PRODUCT"     prd
                       WHERE    prd.curr_rcrd_flag     = 'Y' 
                       AND      prd.prod_id IS NOT NULL
                       QUALIFY ROW_NUMBER() OVER (PARTITION BY prod_key, glb_supply_src_name ORDER BY SRC_RCRD_UPD_DTE DESC) = 1
                      ) prod LEFT JOIN 
                      prod_edw.dimensions.dim_product_cost cst ON cst.prod_key = prod.prod_key 
               AND    (
                      (
                          cst.src_sys_key       = 'SAPC11' 
                      AND cst.cost_variant_lkey = 'ZPC7' 
                      AND cst.cost_eff_to_dte   >= CURRENT_TIMESTAMP
                      AND cst.cost_cmpnt_strc   = '04'
                      AND cst.cost_entr_flag    = 'N'
                      )
                      OR
                      (
                          cst.src_sys_key = 'SAPE03' 
                      AND cst.cost_eff_to_dte >= CURRENT_TIMESTAMP
                      AND cst.cost_entr_flag = 'N'
                      )
                      ) 
               WHERE    prod.prod_key       IN ({material})
               QUALIFY  ROW_NUMBER() OVER (PARTITION BY cst.prod_key, cst.loc_key ORDER BY cst.cost_eff_from_dte DESC) = 1
               UNION ALL
           """.replace("\t"," ").replace("\n"," ")
        consolidated_query_string += sql
    consolidated_query_string = consolidated_query_string[ : consolidated_query_string.rfind("UNION ALL")]
    return consolidated_query_string

In [14]:
def get_bom_cost():
    
    """
    This function extracts the costs associated with an item. The data is saved in bom_cost.csv.
    Cost information is fetched from Snowflake database.
    """
    
    global df_bom_cost
    global df_exploded_bom_cost
    
    access_token = get_oauth_token(svc_username = userid, svc_password = password)
    activesnowflakeconnector = snowflake.connector.connect(account = 'sbd_caspian.us-east-1', 
                                                           authenticator = 'oauth', 
                                                           token = access_token,
                                                           warehouse = 'DEV_AIDA_WH',
                                                           database = 'PROD_EDW',
                                                           role = 'OPERATIONS_CREATOR_RO',
                                                           schema = 'CONSOLIDATED'
                                                          )
    cur = activesnowflakeconnector.cursor()
    
    df_bom_cost = pd.DataFrame(cur.execute(build_bom_cost_query(df_bom, 'bom_material')))
    df_bom_cost.columns = [col[0] for col in cur.description]
    save_data(df = df_bom_cost, file_name = 'bom_cost.csv')
    '''
    df_exploded_bom_cost = pd.DataFrame(cur.execute(build_bom_cost_query(df_exploded_bom, 'component')))
    df_exploded_bom_cost.columns = [col[0] for col in cur.description]
    save_data(df = df_exploded_bom_cost, file_name = 'exploded_bom_cost.csv')
    '''

In [15]:
get_bom_cost()

In [44]:
def build_bom_dimension_query():
    """
    This function builds the query to fetch BOM products' dimensions and 
    how many products can be placed in a 40 in HC or 53 inch trailer without a pallet .
    """
    global df_prod_dim
    
    consolidated_query_string = ""
    for material in df_bom['bom_material'].values:
        sql = f"""
               SELECT prod_id
                    , src_sys_key
                    , len
                    , hgt
                    , wdth
                    , sz_uom_key
                    , vol
                    , vol_uom_key
                    , wgt
                    , wgt_uom_key
                    
                    , 76.39885 AS hc40_container_vol_in_cubic_meter
                    , 28564.98 AS hc40_container_weight_in_kg
                    , 2698     AS hc40_container_vol_in_cubic_ft
                    , 62975    AS hc40_container_weight_in_lb
                    , 0.9      AS max_utilization_factor
                    
                    , CASE WHEN UPPER(wgt_uom_key) = 'LB' THEN (hc40_container_weight_in_lb * max_utilization_factor) / NULLIFZERO(wgt)
                           WHEN UPPER(wgt_uom_key) = 'KG' THEN (hc40_container_weight_in_kg * max_utilization_factor) / NULLIFZERO(wgt)
                      END  AS   hc40_unit1
                    , CASE WHEN UPPER(vol_uom_key) = 'CUB' THEN (hc40_container_vol_in_cubic_ft * max_utilization_factor) / NULLIFZERO(vol)
                           WHEN UPPER(vol_uom_key) IN ('CCM','CDM', 'M3') THEN (hc40_container_vol_in_cubic_meter * max_utilization_factor) / NULLIFZERO(vol)
                      END  AS   hc40_unit2
                    , CASE WHEN NVL(hc40_unit1, 0) > 0 AND NVL(hc40_unit2, 0) > 0 THEN LEAST(hc40_unit1, hc40_unit2) 
                           WHEN NVL(hc40_unit1, 0) = 0 AND NVL(hc40_unit2, 0) > 0 THEN hc40_unit2
                           WHEN NVL(hc40_unit1, 0) > 0 AND NVL(hc40_unit2, 0) = 0 THEN hc40_unit1
                      END      AS hc40_units_to_fit_in_container_wo_pallet 
                    , ROUND(hc40_units_to_fit_in_container_wo_pallet) AS rnd_hc40_units_to_fit_in_container_wo_pallet
                    
                    , 96.27728 AS trl53in_container_vol_in_cubic_meter
                    , 1542.214 AS trl53in_container_weight_in_kg
                    , 3400     AS trl53in_container_vol_in_cubic_ft
                    , 44000    AS trl53in_container_weight_in_lb
                    , CASE WHEN UPPER(wgt_uom_key) = 'LB' THEN (trl53in_container_weight_in_lb * max_utilization_factor) / NULLIFZERO(wgt)
                           WHEN UPPER(wgt_uom_key) = 'KG' THEN (trl53in_container_weight_in_kg * max_utilization_factor) / NULLIFZERO(wgt)
                      END  AS   trl53in_unit1
                    , CASE WHEN UPPER(vol_uom_key) = 'CUB' THEN (trl53in_container_vol_in_cubic_ft * max_utilization_factor) / NULLIFZERO(vol)
                           WHEN UPPER(vol_uom_key) IN ('CCM','CDM', 'M3') THEN (trl53in_container_vol_in_cubic_meter * max_utilization_factor) / NULLIFZERO(vol)
                      END  AS   trl53in_unit2
                    , CASE WHEN NVL(trl53in_unit1, 0) > 0 AND NVL(trl53in_unit2, 0) > 0 THEN LEAST(trl53in_unit1, trl53in_unit2) 
                           WHEN NVL(trl53in_unit1, 0) = 0 AND NVL(trl53in_unit2, 0) > 0 THEN trl53in_unit2
                           WHEN NVL(trl53in_unit1, 0) > 0 AND NVL(trl53in_unit2, 0) = 0 THEN trl53in_unit1
                      END  AS trl53in_units_to_fit_in_container_wo_pallet  
                    , ROUND(trl53in_units_to_fit_in_container_wo_pallet) AS rnd_trl53in_units_to_fit_in_container_wo_pallet
               
               FROM   "PROD_EDW"."DIMENSIONS"."DIM_PRODUCT"
               WHERE  prod_typ_lkey = 'FERT' 
               AND    prod_id IN ({material})
               AND    (rnd_hc40_units_to_fit_in_container_wo_pallet > 0 OR rnd_trl53in_units_to_fit_in_container_wo_pallet > 0)
               UNION ALL
               """.replace("\t"," ").replace("\n"," ")
        consolidated_query_string += sql
    consolidated_query_string = consolidated_query_string[ : consolidated_query_string.rfind("UNION ALL")]    
    return consolidated_query_string

In [45]:
def get_bom_dim():
    
    """
    This function extracts the dimension associated with an BOM item. The data is saved in bom_dim.csv.
    Product dimension information is fetched from Snowflake database.
    """
    
    global df_prod_dim
    
    access_token = get_oauth_token(svc_username = userid, svc_password = password)
    activesnowflakeconnector = snowflake.connector.connect(account = 'sbd_caspian.us-east-1', 
                                                           authenticator = 'oauth', 
                                                           token = access_token,
                                                           warehouse = 'DEV_AIDA_WH',
                                                           database = 'PROD_EDW',
                                                           role = 'OPERATIONS_CREATOR_RO',
                                                           schema = 'CONSOLIDATED'
                                                          )
    cur = activesnowflakeconnector.cursor()
    
    df_prod_dim = pd.DataFrame(cur.execute(build_bom_dimension_query()))
    df_prod_dim.columns = [col[0] for col in cur.description]
    save_data(df = df_prod_dim, file_name = 'bom_dim.csv')

In [46]:
get_bom_dim()

In [ ]:
'''
%run ./tkinter_file_upload.ipynb
print("File upload completed")
%run ./bom.ipynb
print("BOM info job completed")
%run ./bom_cost.ipynb
print("BOM cost job completed")
%run ./exploded_bom_cost.ipynb
print("Exploded BOM cost job completed")
'''